In [40]:
import numpy as np
import pandas as pd
#
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
#
import plotly.graph_objects as go


# Read in price data
df = pd.read_csv('../../data/df_monthly_returns_complete.csv', index_col='Date')
df = df[df.columns[0:200]]

# Calculate expected returns and sample covariance
mu_0 = expected_returns.mean_historical_return(df, frequency=12)

# Get only tickers with a mean historical return of at least 5% 
optimal_tickers = mu_0[mu_0 > 0.05].index
df_optimal = df[optimal_tickers]

mu = expected_returns.mean_historical_return(df_optimal, frequency=12)
S = risk_models.CovarianceShrinkage(df_optimal, frequency=12).ledoit_wolf() # risk_models.sample_cov, # Ledoit-Wolf shrinkage (df_optimal, frequency=12), # Exponential Covariance

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
ef_new = EfficientFrontier(mu, S)

raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
#
ef.portfolio_performance(verbose=True)

Expected annual return: 16.2%
Annual volatility: 11.7%
Sharpe Ratio: 1.22


(0.16189289251795783, 0.1167161226170067, 1.215709443874917)

In [41]:
S

,RS1.L,HNI,VNA.DE,KEYS,SGRO.L,CBRE,BRC,REL.L,BHE,PLXS,...,SAP.DE,POOL,AZO,LKQ,2154.T,HPQ,NSIT,KMX,OXIG.L,CDNS
RS1.L,0.082105,0.009164,0.005643,0.009874,0.010015,0.015225,0.011476,0.006469,0.010304,0.010412,...,0.010118,0.009902,0.006112,0.010716,0.014472,0.015036,0.009667,0.010899,0.005441,0.008562
HNI,0.009164,0.133516,0.008332,0.005401,0.026115,0.050524,0.040937,0.005948,0.036834,0.044663,...,0.019948,0.040877,0.019306,0.025180,0.027553,0.029881,0.072677,0.056009,0.021288,0.024008
VNA.DE,0.005643,0.008332,0.062092,0.003944,0.008712,0.017502,0.007605,0.004555,0.006397,0.004131,...,0.005030,0.010688,0.001899,0.008859,0.021463,0.011475,0.005654,0.010769,0.003006,0.004789
KEYS,0.009874,0.005401,0.003944,0.062330,0.005879,0.007573,0.009518,0.002627,0.010555,0.010185,...,0.006234,0.009160,0.005156,0.008573,0.004821,0.007151,0.016366,0.014255,0.004979,0.011915
SGRO.L,0.010015,0.026115,0.008712,0.005879,0.088666,0.025557,0.020360,0.011337,0.017786,0.020454,...,0.015402,0.026055,0.009364,0.008266,0.021166,0.019264,0.042595,0.030079,0.014499,0.025011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HPQ,0.015036,0.029881,0.011475,0.007151,0.019264,0.047579,0.029917,0.011562,0.046667,0.057541,...,0.043700,0.021905,0.005433,0.027100,0.039698,0.126331,0.052254,0.035046,0.025768,0.037986
NSIT,0.009667,0.072677,0.005654,0.016366,0.042595,0.058594,0.062635,0.019916,0.072188,0.106610,...,0.047124,0.043835,0.024916,0.027998,0.050889,0.052254,0.258228,0.073450,0.054900,0.061655
KMX,0.010899,0.056009,0.010769,0.014255,0.030079,0.051670,0.040216,0.008540,0.041462,0.052662,...,0.030624,0.045806,0.031693,0.038179,0.036217,0.035046,0.073450,0.231448,0.021678,0.042532
OXIG.L,0.005441,0.021288,0.003006,0.004979,0.014499,0.035708,0.016207,0.015161,0.022468,0.044879,...,0.017470,0.013741,0.003728,0.011700,0.015948,0.025768,0.054900,0.021678,0.169225,0.031133


In [42]:
latest_prices = get_latest_prices(df)

da = DiscreteAllocation(raw_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: €{:.2f}".format(leftover))

Discrete allocation: {'AZO': 1, 'PHP.L': 8, 'TPL': 1, 'BMI': 3, 'POOL': 2, 'DHR': 2, 'TEP.L': 1, 'BALL': 8, 'RGLD': 3, 'CDW': 1, 'LIN': 1}
Funds remaining: €58.57


In [43]:
# Generate efficient frontier data
target_returns = np.linspace(mu.min(), mu.max()-0.05, 50)
frontier_volatility = []
frontier_returns = []
for r in target_returns:
    ab = ef_new.efficient_return(target_return=r)
    ret, vol, _ = ef_new.portfolio_performance()
    frontier_returns.append(ret)
    frontier_volatility.append(vol)

In [44]:
import plotly.graph_objects as go

# Create a Plotly scatter plot
fig = go.Figure()

# Convert expected returns to a NumPy array
returns_array = mu.values

# Calculate volatilities (square root of the diagonal of covariance matrix)
volatilities_array = np.sqrt(np.diag(S))

# Marker: Add annotations for each point
for i, (ret, vol, ticker) in enumerate(zip(returns_array, volatilities_array, mu.keys())):
    fig.add_trace(go.Scatter(
        x=[vol],
        y=[ret],
        mode='markers+text',
        # text=f"{ticker} ({ret:.2%}, {vol:.2%})",
        textposition="top center",
        marker=dict(size=6, color='blue'),
        showlegend=False
    ))

max_sharpe_return, max_sharpe_volatility, _ = ef.portfolio_performance()
# Marker: Add the maximum Sharpe ratio portfolio
fig.add_trace(go.Scatter(
    x=[max_sharpe_volatility],
    y=[max_sharpe_return],
    mode='markers',
    name='Max Sharpe Portfolio',
    marker=dict(color='red', size=10, symbol='star')
))

# Line: Add the efficient frontier
fig.add_trace(go.Scatter(
    x=frontier_volatility,
    y=frontier_returns,
    mode='lines',
    name='Efficient Frontier',
    line=dict(color='blue', width=3)
))

fig.update_traces(hovertemplate=None)
# Customize layout
fig.update_layout(
    title='Efficient Frontier',
    xaxis=dict(
        title='Risk (Standard Deviation)',
        tickformat='.0%',
    ),
    yaxis=dict(
        title='Investment Return Average (last 5 years)',
        tickformat='.0%',
    ),
    legend=dict(x=1, y=1),
    hovermode='closest',
    template='plotly'
)

fig.show()